In [54]:
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
bp = pd.read_csv('data/Building_Permits_Issued.csv')

/home/michael/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [24]:
bp.head()

,Permit #,Permit Type Description,Permit Subtype Description,Parcel,Date Entered,Date Issued,Const. Cost,Address,City,State,Zip,Subdivision/Lot,Contact,Permit Type,Permit Subtype,IVR Trk#,Purpose,Council Dist,Census Tract,Mapped Location
0,2018002927,Building Residential - New,Single Family Residence,081160R00100CO,01/18/2018,02/12/2018,206150.0,1017A MONROE ST,NASHVILLE,TN,37208,UNIT A 1017 MONROE,"TRISTAR CONSTRUCTION, INC",CARN,CAA01R301,3439281,to construct 1830SF single family residence wi...,19.0,37019400.0,"1017A MONROE ST\nNASHVILLE, TN 37208\n(36.1740..."
1,201617929,Building Use & Occupancy,"Tents, Stages",09215014800,04/19/2016,04/21/2016,2000.0,2300 WEST END AVE,NASHVILLE,TN,37203,N W CORNER WEST END AVE & 23RD AVE N,Music City Tents & Events Llc,CAUO,CAZ06A001,2116751,PERMIT TO ERECT 1:40X85 TENT. THIS TENT IS BEI...,21.0,37016500.0,"2300 WEST END AVE\nNASHVILLE, TN 37203\n(36.14..."
2,2016049270,Building Sign Permit,Sign - Ground / Wall Signs,11909014000,09/14/2016,09/14/2016,15000.0,2802 NOLENSVILLE PIKE,NASHVILLE,TN,37211,PT LOT 2 PATTERSON SUB,BOZMAN SIGN CO INC,CASN,CAZ02A001,3277064,to install replacement signage for existing fr...,16.0,37017300.0,"2802 NOLENSVILLE PIKE\nNASHVILLE, TN 37211\n(3..."
3,2016029907,Building Residential - Roofing / Siding,Duplex,075150A13900CO,06/20/2016,06/20/2016,17648.0,139 STONERS GLEN DR,HERMITAGE,TN,37076,UNIT 139 STONERS GLEN PHASE 1,"MR ROOF NASHVILLE, LLC",CARK,CAA02R302,3251947,Remove existing shingles and replace with new ...,12.0,37015623.0,"139 STONERS GLEN DR\nHERMITAGE, TN 37076\n(36...."
4,201533533,Building Residential - Tenant Finish Out,"Multifamily, Tri-Plex, Quad, Apartments",10404024100,08/07/2015,11/17/2015,542696.0,70 MUSIC SQ W,NASHVILLE,TN,37203,LOT 1 O.B. HAYES FIRST PLAN CONSOLIDATION OF L...,Construction Enterprises Inc,CART,CAA03R398,2061885,SEE MASTER PERMIT T2015-26520...T.....GRAND AV...,19.0,37016400.0,"70 MUSIC SQ W\nNASHVILLE, TN 37203\n(36.147074..."


In [3]:
building = bp.loc[bp['Permit Type Description'].str.contains('Building')]

In [4]:
new_building = building.loc[building['Permit Type Description'].str.contains('New')]

In [5]:
demolition = bp.loc[bp['Permit Type Description'].str.contains('Demolition')]

In [6]:
res_demolition = demolition.loc[demolition['Permit Subtype Description'].str.contains('Residential')]

In [7]:
merged = pd.merge(new_building, res_demolition, on= 'Address')

In [14]:
bp.loc[bp['Permit Subtype'].isin(['CAZ01A001', 'CAZ01A002'])];

In [11]:
bp.loc[bp['Permit Subtype'] == 'CAZ01A001']

,Permit #,Permit Type Description,Permit Subtype Description,Parcel,Date Entered,Date Issued,Const. Cost,Address,City,State,Zip,Subdivision/Lot,Contact,Permit Type,Permit Subtype,IVR Trk#,Purpose,Council Dist,Census Tract,Mapped Location
23,201543896,Building Demolition Permit,Demolition Permit - Residential,10509038300,10/08/2015,10/08/2015,4000.0,914 ACKLEN AVE,NASHVILLE,TN,37203,LOT 2 WAVERLY PLACE RESUB PT. LOT 31,Graham Construction Of Tennessee Llc,CADM,CAZ01A001,2075785,Work on this demolition permit must commence w...,17.0,37016200.0,"914 ACKLEN AVE\nNASHVILLE, TN 37203\n(36.13354..."
30,201620413,Building Demolition Permit,Demolition Permit - Residential,08116022300,05/03/2016,05/03/2016,5000.0,918 SCOVEL ST,NASHVILLE,TN,37208,LOT 55 MCGAVOCK TOWN NORTH NASHVILLE,K Jordan Construction Llc,CADM,CAZ01A001,2120019,Work on this demolition permit must commence w...,19.0,37019400.0,"918 SCOVEL ST\nNASHVILLE, TN 37208\n(36.173632..."
36,201614474,Building Demolition Permit,Demolition Permit - Residential,13102006500,03/31/2016,03/31/2016,7500.0,1923 WARFIELD DR,NASHVILLE,TN,37215,LOT 21 COLONIAL HGTS,Cat3 Restoration Llc,CADM,CAZ01A001,2112314,to demolish exisitng single family residence. ...,25.0,37017901.0,"1923 WARFIELD DR\nNASHVILLE, TN 37215\n(36.102..."
37,2016039565,Building Demolition Permit,Demolition Permit - Residential,10200007600,08/03/2016,08/03/2016,21000.0,3985 OLD HICKORY BLVD,NASHVILLE,TN,37218,S/S OF OLD HICKORY BV E OF HYDES FERRY PK,SELF CONTRACTOR COMMERCIAL (SEE APPLICANT INFO...,CADM,CAZ01A001,3264459,to demolish existing single family residence. ...,1.0,37013100.0,"3985 OLD HICKORY BLVD\nNASHVILLE, TN 37218\n(3..."
47,2017033484,Building Demolition Permit,Demolition Permit - Residential,15900008000,05/19/2017,05/19/2017,10000.0,5610 GRANNY WHITE PIKE,BRENTWOOD,TN,37027,LOTS 1 & 2 GRANNY WHITE PARTNERS,"URBAN DEVELOPMENT GROUP, LLC",CADM,CAZ01A001,3368029,to demolish existing residence. not to be bur...,34.0,37018700.0,"5610 GRANNY WHITE PIKE\nBRENTWOOD, TN 37027\n(..."
61,2016066262,Building Demolition Permit,Demolition Permit - Residential,10501054100,11/29/2016,11/29/2016,3000.0,1004 SOUTHSIDE AVE,NASHVILLE,TN,37203,LOT 38 SOUTHSIDE PARK EST SEC 1,"MILLER, B. RECYCLING, LLC",CADM,CAZ01A001,3298215,Work on this demolition permit must commence w...,17.0,37016200.0,"1004 SOUTHSIDE AVE\nNASHVILLE, TN 37203\n(36.1..."
67,2016051094,Building Demolition Permit,Demolition Permit - Residential,08400000600,09/22/2016,09/22/2016,2500.0,0 EASTLAND AVE,NASHVILLE,TN,37206,RESERVED PARCEL TIFFANY TERRACE,FRITSCH DESIGN BUILD LLC,CADM,CAZ01A001,3279260,THIS PERMIT IS TO DEMOLISH EXISTING BARN. NOT...,6.0,37011600.0,"0 EASTLAND AVE\nNASHVILLE, TN 37206\n"
82,2016025907,Building Demolition Permit,Demolition Permit - Residential,10505038400,06/01/2016,06/02/2016,8000.0,1014 ARGYLE AVE,NASHVILLE,TN,37203,LOT 8 BRANSFORD RLTY CO SUB JONES & DUDLEY,LANDMARK HOMES OF TN INC,CADM,CAZ01A001,3246625,Demolition of building on this lot,17.0,37016200.0,"1014 ARGYLE AVE\nNASHVILLE, TN 37203\n(36.1375..."
105,2018000996,Building Demolition Permit,Demolition Permit - Residential,072164F90000CO,01/05/2018,01/05/2018,4000.0,1507C WAYNE DR,NASHVILLE,TN,37206,COMMON AREA 1507 WAYNE DRIVE TOWNHOMES,"CHRISTIAN, MICHAEL J HOUSEMOVING",CADM,CAZ01A001,3436815,to demolish existing residence. not to be bur...,7.0,37011600.0,"1507C WAYNE DR\nNASHVILLE, TN 37206\n(36.1938,..."
120,2016031224,Building Demolition Permit,Demolition Permit - Residential,08203030300,06/24/2016,06/24/2016,4000.0,1000 PENNOCK AVE,NASHVILLE,TN,37207,PT LOT 10 F E MANN PLAN,"JAMAAR GROUP CONSTRUCTION, LLC",CADM,CAZ01A001,3253609,to demolish existing residence. not to be bur...,5.0,37011800.0,"1000 PENNOCK AVE\nNASHVILLE, TN 37207\n(36.188..."


In [12]:
new_commercial = ['CAA03R299', 'CAA03R301', 'CAA03R398', 'CAA03R399', 'CAA05R202']

In [27]:
new_residential = ['CAA01R301', 'CAA02R302', 'CAA03R298', 'CAA03R299', 'CAA03R301', 'CAA03R398', 'CAA03R399', 'CAA04R301', 'CAA05R202', 'CAA08R301', 'CAA10R301', 'CAZ08A001', 'DPLX2CNDO']

In [28]:
matched = pd.merge(bp.loc[bp['Permit Subtype'].isin(['CAZ01A001', 'CAZ01A002'])], new_building.loc[new_building['Permit Subtype'].isin(new_commercial + new_residential)], on = 'Address')

In [29]:
matched.shape

(1060, 39)

In [30]:
matched.loc[0]

Permit #_x                                                             2016025907
Permit Type Description_x                              Building Demolition Permit
Permit Subtype Description_x                      Demolition Permit - Residential
Parcel_x                                                              10505038400
Date Entered_x                                                         06/01/2016
Date Issued_x                                                          06/02/2016
Const. Cost_x                                                                8000
Address                                                          1014  ARGYLE AVE
City_x                                                                  NASHVILLE
State_x                                                                        TN
Zip_x                                                                       37203
Subdivision/Lot_x                      LOT 8 BRANSFORD RLTY CO SUB JONES & DUDLEY
Contact_x       

In [31]:
sf2009 = pd.read_csv('data/2009SINGLEFAMILY.txt')

In [32]:
sf2013 = pd.read_csv('data/2013SINGLEFAMILY.txt')
sf2017 = pd.read_csv('data/2017SINGLEFAMILY.txt')

In [36]:
sf = pd.merge(pd.merge(sf2009, sf2013, on = 'AddressFullAddress'), sf2017, on = 'AddressFullAddress')

In [48]:
sf2009.head()

,APN,DistrictCode,Council District,AddressFullAddress,AddressCity,AddressPostalCode,2009 LAND,2009 IMPR,2009 TOTAL APPR,2009 TOTAL ASSD
0,00200000300,RT,10,1012 OLD BUCK HILL RD,GOODLETTSVILLE,37072,49800.0,65700.0,115500.0,28875.0
1,00200000700,RT,10,1109 HIGHLAND AVE,GOODLETTSVILLE,37072,44100.0,88700.0,132800.0,33200.0
2,00200001001,GSD,10,1992 SPRINGFIELD HWY,GOODLETTSVILLE,37072,0.0,0.0,0.0,0.0
3,00200001001002,GSD,10,0 SPRINGFIELD HWY,GOODLETTSVILLE,37072,15000.0,137900.0,152900.0,38225.0
4,00200001200,GSD,10,1855 LAKE RD,RIDGETOP,37152,0.0,0.0,0.0,0.0


In [58]:
dummy = pd.merge(matched, sf, left_on = 'Address', right_on= 'AddressFullAddress')

In [59]:
dummy.shape

(41, 67)

In [60]:
sf.shape

(197254, 28)

In [61]:
matched.shape

(1060, 39)

In [66]:
sf.loc[sf.AddressFullAddress == '1014  ARGYLE AVE']

,APN_x,DistrictCode_x,Council District_x,AddressFullAddress,AddressCity_x,AddressPostalCode_x,2009 LAND,2009 IMPR,2009 TOTAL APPR,2009 TOTAL ASSD,...,2013 TOTAL ASSD,APN,DistrictCode,Council District,AddressCity,AddressPostalCode,2017 LAND,2017 IMPR,2017 TOTAL APPR,2017 TOTAL ASSD
88950,10505038400,USD,17,1014 ARGYLE AVE,NASHVILLE,37203,72000.0,53400.0,125400.0,31350.0,...,33000.0,10505038400,USD,17,NASHVILLE,37203,176000,216600,392600,98150


In [68]:
matched.Address = matched.Address.str.replace(' ', '')

In [69]:
sf.AddressFullAddress = sf.AddressFullAddress.str.replace(' ', '')

In [70]:
dummy = pd.merge(matched, sf, left_on = 'Address', right_on= 'AddressFullAddress')

In [71]:
dummy.shape

(629, 67)

In [73]:
dummy.loc[0]

Permit #_x                                                             2016025907
Permit Type Description_x                              Building Demolition Permit
Permit Subtype Description_x                      Demolition Permit - Residential
Parcel_x                                                              10505038400
Date Entered_x                                                         06/01/2016
Date Issued_x                                                          06/02/2016
Const. Cost_x                                                                8000
Address                                                             1014ARGYLEAVE
City_x                                                                  NASHVILLE
State_x                                                                        TN
Zip_x                                                                       37203
Subdivision/Lot_x                      LOT 8 BRANSFORD RLTY CO SUB JONES & DUDLEY
Contact_x       

In [74]:
sf['Pct_Change'] = (sf['2017 TOTAL APPR'] - sf['2013 TOTAL APPR']) / sf['2013 TOTAL APPR']

In [79]:
sf.Pct_Change.describe()

count    197254.000000
mean          0.428449
std           0.441337
min          -0.967016
25%           0.229376
50%           0.346133
75%           0.487931
max          58.333333
Name: Pct_Change, dtype: float64

In [80]:
dummy['Pct_Change'] = (dummy['2017 TOTAL APPR'] - dummy['2013 TOTAL APPR']) / dummy['2013 TOTAL APPR']

In [82]:
dummy.Pct_Change.describe()

count    629.000000
mean       1.438354
std        1.716807
min       -0.819005
25%        0.305965
50%        0.854127
75%        2.255941
max       12.150456
Name: Pct_Change, dtype: float64